# Word embedding

In [2]:
import numpy as np
import re
import pandas as pd
from typing import List

## Read data

In [3]:
def read_file(file_name: str) -> list:
    with open(file_name) as file:
         return [word for line in file for word in re.findall(r'\w+', line)]

In [4]:
data = read_file(file_name= 'data.txt')

## Build word-word co-occurrence matrix

In [9]:
def build_co_occurrence_matrix(data: List[str], window_size = (10, 10)) -> np.ndarray:
    words = list(set(data))
    vocab = {word:index for index, word in enumerate(words)}
    number_data = np.array([vocab[word] for word in data])
    
    left_window =  np.arange(start= 1, stop= window_size[0] + 1, step= 1)
    right_window =  np.arange(start= 1, stop= window_size[1] + 1, step= 1)
    indices = np.arange(start= 0, stop= len(number_data), step= 1).reshape(-1, 1)
    
    left_windows = np.take(number_data, indices[window_size[0]:] - left_window)
    right_windows = np.take(number_data, indices[:-window_size[1]] + right_window)
    
    
    number_of_words = len(words)
    co_occurrence_matrix = np.zeros(shape= (number_of_words, number_of_words), dtype= np.uint64)
    co_occurrence_matrix[np.repeat(number_data[window_size[0]:], window_size[0]), left_windows.ravel()] += 1
    co_occurrence_matrix[np.repeat(number_data[:-window_size[1]], window_size[1]), right_windows.ravel()] += 1
  

    #1. adaugare distanta fata de cuvantul curent
    co_occurrence_matrix += 1

    return pd.DataFrame(data = co_occurrence_matrix,
                       index = words,
                       columns= words), co_occurrence_matrix

In [10]:
dataframe, co_occurrence = build_co_occurrence_matrix(data, window_size=(2, 2))

### Check if co-occurence matrix is symmetric

In [11]:
print(f'Symmetric: {np.all(co_occurrence[1] == co_occurrence[1].T)}')

Symmetric: True


In [12]:
print(f'Max value: {np.max(co_occurrence)}')
print(f'Min value: {np.min(co_occurrence)}')    

Max value: 3
Min value: 1


In [13]:
dataframe

,porta,sem,Aliquam,nec,Quisque,sapien,vehicula,aptent,torquent,eget,...,dictum,convallis,arcu,nisi,aliquet,et,sollicitudin,Maecenas,turpis,habitant
porta,1,3,2,2,2,2,2,1,1,3,...,2,1,3,3,2,2,2,1,3,2
sem,3,1,2,3,3,1,3,1,1,2,...,2,2,2,1,3,3,3,2,3,1
Aliquam,2,2,1,1,1,1,2,1,1,3,...,3,3,2,3,2,1,3,1,2,1
nec,2,3,1,3,2,3,3,1,1,2,...,3,3,3,3,2,3,3,2,3,1
Quisque,2,3,1,2,1,2,3,1,1,2,...,2,1,1,2,1,1,2,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
et,2,3,1,3,1,2,3,1,1,3,...,2,2,3,3,3,3,3,3,2,1
sollicitudin,2,3,3,3,2,3,2,1,1,3,...,2,2,2,1,3,3,3,3,2,1
Maecenas,1,2,1,2,1,3,3,1,1,3,...,2,2,3,2,2,3,3,1,3,1
turpis,3,3,2,3,3,1,2,1,1,3,...,3,2,1,3,2,2,2,3,1,1


## Build context probability matrix

In [14]:
def build_probability_matrix(co_occurrence_matrix: np.ndarray) -> np.ndarray:
    return co_occurrence_matrix / np.sum(co_occurrence_matrix, axis= 1, keepdims= True)

In [15]:
probability = build_probability_matrix(co_occurrence)

In [16]:
print(f'Probability matrix correctness: {np.all(np.isclose(np.sum(probability, axis = 1), 1))}')

Probability matrix correctness: True


In [17]:
print(probability)

[[0.00276243 0.00828729 0.00552486 ... 0.00276243 0.00828729 0.00552486]
 [0.00797872 0.00265957 0.00531915 ... 0.00531915 0.00797872 0.00265957]
 [0.00578035 0.00578035 0.00289017 ... 0.00289017 0.00578035 0.00289017]
 ...
 [0.00276243 0.00552486 0.00276243 ... 0.00276243 0.00828729 0.00276243]
 [0.00771208 0.00771208 0.00514139 ... 0.00771208 0.00257069 0.00257069]
 [0.00934579 0.0046729  0.0046729  ... 0.0046729  0.0046729  0.0046729 ]]


## Model

### Notations

$ w \to \verb|word| $

$ V \to \verb|number of unique words - size of vocabulary| $

$ X \to \verb|word-word co-occurrence matrix, symmetrically built - | X_{ij} = X_{ji} $

$ X_{ij} \to \verb|the number of times word j occurs in the context of word i| $

$ X_{i} = \sum_{k}{X_{k}} \to \verb|the number of times any word appears in the context of word i| $

$ P_{ij} = P\left( j\mid i \right) = \frac{X_{ij}}{X_{i}} \to \verb|the probability that word j appear in the context of word i| $

### Word representation

$ w_{i}^{T}w_{j} = \log{P\left( j\mid i \right)} \to w_{i} \verb| and | w_{j} \verb|unknown vectors that represents i-th, j-th words| $

$ w_{i}^{T}w_{j} = \log{X_{ij}} - \log{X_{i}} $

$ w_{j}^{T}w_{i} = \log{X_{ij}} - \log{X_{j}} \quad (X_{ij} = X_{ji})$

$ 2 w_{i}^{T}w_{j} = 2\log{X_{ij}} - 2\log{X_{i}} - 2\log{X_{j}} $

$ w_{i}^{T}w_{j} = \log{X_{ij}} - \frac{1}{2}\log{X_{i}} - \frac{1}{2}\log{X_{j}} $

$ w_{i}^{T}w_{j} = \log{X_{ij}} - b_{i} -  b_{j} $

$ w_{i}^{T}w_{j} + b_{i} + b_{j} = \log{X_{ij}}  $

### Cost function

$ J = \sum_{i = 1}^{V}\sum_{j = 1}^{V} f\left( X_{i,j} \right){\left(w_{i}^{T}\tilde{w_{j}}+b_{i}+\tilde{b_{j}}-\log{X_{ij}}\right)}^2 $

### Learning

In [239]:
class MatrixFactorization(object):

    def __init__(self, matrix: np.ndarray):
        self.__matrix = matrix
        
    def train(self,
              desired_dimensions: int,
              alpha: float= 1e-2,
              beta: float= 1e-2,
              epsilon= 1e-5,
              iterations: int= 1e3,
              auto_adjust_initial_values= True):
        self.U = np.random.rand(self.__matrix.shape[0], desired_dimensions)
        self.V = np.random.rand(self.__matrix.shape[1], desired_dimensions)
        self.b = np.zeros(self.__matrix.shape)
        
        if auto_adjust_initial_values:
            mean = np.mean(self.__matrix)
            self.U *= mean
            self.V *= mean
            self.b *= mean
        
        return self.__gradient_descent(alpha, beta, epsilon, iterations)
       
    def get_predicted_matrix(self):
        return self.U@self.V.T + self.b
    
    def get_vector(self, line: int, column: int) -> np.ndarray:
        return self.U[line] * self.V[column]

    def __gradient_descent(self, alpha, beta, epsilon, iterations) -> np.ndarray:
        current_step = 0
        errors = []
        while True:
            error = self.__matrix - self.get_predicted_matrix()
            old_U = self.U.copy()
            
            self.b += alpha * (error - beta * self.b)
            
            self.U += alpha * (error @ self.V - beta * self.U)
            self.V += alpha * (error.T @ old_U - beta * self.V)
            errors.append(self.__compute_mean_squared_error())
            current_step += 1
            
            if np.linalg.norm(self.U - old_U) < epsilon or current_step > iterations:
                return {
                    'errors' : errors,
                    'iterations' : current_step
                }
    
    def __compute_mean_squared_error(self) -> np.ndarray:
        return np.mean((self.__matrix - self.get_predicted_matrix()) ** 2)

In [240]:
mf2 = MatrixFactorization(matrix= probability)

In [260]:
mf2.train(desired_dimensions= 100)

{'errors': [2.2444798759950856e-05,
  2.1900990338573247e-05,
  2.1368825362196475e-05,
  2.0848093419811594e-05,
  2.0338587505570276e-05,
  1.984010393930549e-05,
  1.935244229000097e-05,
  1.8875405302144634e-05,
  1.840879882489949e-05,
  1.795243174402444e-05,
  1.7506115916475754e-05,
  1.7069666107619077e-05,
  1.6642899930980492e-05,
  1.6225637790464283e-05,
  1.5817702824964176e-05,
  1.5418920855293935e-05,
  1.5029120333362713e-05,
  1.4648132293519932e-05,
  1.4275790305994087e-05,
  1.391193043234965e-05,
  1.355639118288609e-05,
  1.3209013475903028e-05,
  1.2869640598755663e-05,
  1.2538118170624955e-05,
  1.2214294106927297e-05,
  1.189801858528908e-05,
  1.158914401301209e-05,
  1.1287524995956548e-05,
  1.0993018308769376e-05,
  1.0705482866386407e-05,
  1.042477969673826e-05,
  1.015077191459092e-05,
  9.883324696453327e-06,
  9.62230525648572e-06,
  9.367582823344023e-06,
  9.119028617897117e-06,
  8.876515831755549e-06,
  8.639919606551995e-06,
  8.409117013915545

In [261]:
mf2.get_predicted_matrix()

array([[0.00274688, 0.00821698, 0.00548143, ..., 0.00274663, 0.00821758,
        0.00547697],
       [0.00791102, 0.00264527, 0.00527773, ..., 0.00527782, 0.00791224,
        0.00264013],
       [0.00573451, 0.00573476, 0.00287282, ..., 0.00287306, 0.0057353 ,
        0.00286837],
       ...,
       [0.00274672, 0.00548233, 0.00274629, ..., 0.0027468 , 0.00821733,
        0.00274165],
       [0.00764732, 0.00764743, 0.00510166, ..., 0.00764717, 0.00255772,
        0.00255211],
       [0.00926428, 0.00463783, 0.00463719, ..., 0.00463763, 0.00463853,
        0.00463289]])

In [262]:
probability

array([[0.00276243, 0.00828729, 0.00552486, ..., 0.00276243, 0.00828729,
        0.00552486],
       [0.00797872, 0.00265957, 0.00531915, ..., 0.00531915, 0.00797872,
        0.00265957],
       [0.00578035, 0.00578035, 0.00289017, ..., 0.00289017, 0.00578035,
        0.00289017],
       ...,
       [0.00276243, 0.00552486, 0.00276243, ..., 0.00276243, 0.00828729,
        0.00276243],
       [0.00771208, 0.00771208, 0.00514139, ..., 0.00771208, 0.00257069,
        0.00257069],
       [0.00934579, 0.0046729 , 0.0046729 , ..., 0.0046729 , 0.0046729 ,
        0.0046729 ]])

In [263]:
np.isclose(mf2.get_predicted_matrix(), probability, rtol= 0.0001, atol= 0.0001)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [267]:
np.sum(np.isclose(mf2.get_predicted_matrix(), probability, rtol= 0.0001, atol= 0.0001) == False)/ \
probability.size

0.000252518875785965

In [268]:
np.sum(np.isclose(mf2.get_predicted_matrix(), probability, rtol= 0.0001, atol= 0.0001) == False)

10